In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNI

In [8]:
%ls /scratch1/jhoff/checkpoints/

generator_pythia-160M/     reward_pythia-1B-Deduped_peft/
generator_pythia-1B/       reward_pythia-2.8B-Deduped_peft/
reward-model_pythia-160M/


In [24]:
model_path = '/scratch1/jhoff/checkpoints/generator_pythia-1B/runs/step_8000_merged'
#model_path = "EleutherAI/pythia-2.8B-Deduped"
model = GPTNeoXForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/Pythia-1B")
model.config.pad_token_id = model.config.eos_token_id
model = model.cuda().eval()

In [25]:
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)
pipeline

In [26]:
generation_kwargs = {
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    # "eos_token_id": 100_000,
    "min_length": 32,
    "max_length": 128,
}

questions = [
    "ELI5: What is 1+1?",
    # "Who is the president of the U.S.?",
    # "Why is the sky blue?", 
    # "Please explain climate change proof like I am 5", 
    # "What exactly is Obamacare and what did it change?", 
    'ELI5: The 3:1 microphone rule', 
    "ELI5: why are humans considered three dimensional beings when we also move through the fourth dimension of time?\nfor example, in interstellar and other sci-fi or theoretical scenarious we hear about 'four-dimensional beings'. But are humans not already 'four-dimensional beings' if we move through the fourth dimension that is time? ", 
    'ELI5: Would there be any legal ramifications if a major company were to use an idea from a website like Reddit?\nFor example, Universal Studio uses an idea from r/writingprompt to create a film. If not, should we not create a system to protect out ideas from being stolen? ', 
    "ELI5: What's the difference between blu-ray quality and 1080p? \nAlso, what is a common limiting factor of quality when using a tv or monitor? ", 
    'ELI5: Why does blood taste like metal? ', 
    'ELI5: Why do all of the new hires at my job suck? ', 
    'Eli5 Best yo mama jokes please? ', 
    "Eli5: Why can people sue Ashley Madison even though they weren't the ones that released the information? ", 
    'ELI5: Toyota can\'t make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copying? ', 
]      

for question in questions:
    result = pipeline(f"""Question: {question}\n\n\nAnswer:""", **generation_kwargs)
    result = result[0]['generated_text']
    print('-' * 30)
    print(f'{result}')
    print('-')
    print(f"Output length: {len(result)}")
    print(f"Output tokens: {tokenizer(result)['input_ids'].__len__()}")

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: ELI5: What is 1+1?


Answer: Unicast ECM (Encapsulated Multi-Protocol)


Evidence ELI5: Design Anchoring Annotations for Multidomain Replicated Data (Support and Development)

Description: This paper covers design and processing, and uses parallelism for
multidimensional reusable reference files


Faculty : Third author

Dates:


Jobs :


Aide :


This is a PDF. You can embed this file into your post, but not download it.
Just a reminder that the changes
-
Output length: 468
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: ELI5: The 3:1 microphone rule


Answer: For short, no.

ELI5: There are not a lot of ways to fool a good microphone. If they can catch
you, they can probably expand a good environment for you. A competitor needs to
have the best autofocus, and also have one where you can control it.


Join the ELI5 Team on GitHub!



LoudP.Audio (LoudP.Audio)
http://licaloudbytes.github.com/

Ben Loudon (LoudP.Audio)
-
Output length: 413
Output tokens: 128


...

------------------------------
Question: ELI5: Why do all of the new hires at my job suck? 


Answer: We now have a dual position on the north campus, but
that increases the number of students that serve as fill's
contingent North West State University transfer doctor, so it is
to our benefit to allow LTI and University Hall to work with
these students. This is especially true when they need to submit
their coursework for credit transfer purposes, given their
informat[ion of a tragic incident].


Specifically, we believe that these new faculty who have served
on campus represent
-
Output length: 554
Output tokens: 127


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: Eli5 Best yo mama jokes please? 


Answer:  I never heard of that.

Eli5, 

So my mother lost so much weight but she can still blow catcalls, 
is she great? 

Answer: She is.   

Elise, 


So my boyfriend Jamie kisses all the girls in Tyson's Corner and 
their mothers caused them to gain more weight but they are okay... 

Is that why but stop.  

Answer:  They don't have breasts.  (Glasses running over his petunia)


-
Output length: 430
Output tokens: 127


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: Eli5: Why can people sue Ashley Madison even though they weren't the ones that released the information? 


Answer:

It is interesting to note that, for many years, Saudi or Shia
  aliens have been jailed for committing an actual crime and have not sued for it.
  Now, this doesn't necessarily mean they're bad people as most don't leave
  themselves open to lawsuits anymore as they don't want other people to
  know of their wrongdoing. However, in these cases it does show me that
  they'll suffer the consequences. In the end, however
-
Output length: 548
Output tokens: 128
------------------------------
Question: ELI5: Toyota can't make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copying? 


Answer:

Although YouTuber Britney Riedel explored this in an April

In [7]:
model.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-2.8B-Deduped",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.0.dev0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

In [1]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

from reddit_dataset import load_reddit_dataset

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/scratch1/jhoff/checkpoints/reward_pythia-1B-Deduped_peft/checkpoint-250000_adapter
